In [1]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import wooldridge
from scipy.stats import t

In [2]:
# データの読み込み
cps = wooldridge.data('CPS78_85')

In [3]:
# データの内容の確認
wooldridge.data('CPS78_85', description=True)

name of dataset: cps78_85
no of variables: 15
no of observations: 1084

+----------+-----------------------+
| variable | label                 |
+----------+-----------------------+
| educ     | years of schooling    |
| south    | =1 if live in south   |
| nonwhite | =1 if nonwhite        |
| female   | =1 if female          |
| married  | =1 if married         |
| exper    | age - educ - 6        |
| expersq  | exper^2               |
| union    | =1 if belong to union |
| lwage    | log hourly wage       |
| age      | in years              |
| year     | 78 or 85              |
| y85      | =1 if year == 85      |
| y85fem   | y85*female            |
| y85educ  | y85*educ              |
| y85union | y85*union             |
+----------+-----------------------+

Professor Henry Farber, now at Princeton University, compiled these
data from the 1978 and 1985 Current Population Surveys. Professor
Farber kindly provided these data when we were colleagues at MIT.


In [4]:
# 回帰分析

formula = 'lwage ~ y85 + educ + female + \
                   y85:educ + y85:female + \
                   exper + I((exper**2)/100) + union'

result = ols(formula, cps).fit()

In [5]:
print(result.summary().tables[1])

                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.4589      0.093      4.911      0.000       0.276       0.642
y85                       0.1178      0.124      0.952      0.341      -0.125       0.361
educ                      0.0747      0.007     11.192      0.000       0.062       0.088
female                   -0.3167      0.037     -8.648      0.000      -0.389      -0.245
y85:educ                  0.0185      0.009      1.974      0.049       0.000       0.037
y85:female                0.0851      0.051      1.658      0.098      -0.016       0.186
exper                     0.0296      0.004      8.293      0.000       0.023       0.037
I((exper ** 2) / 100)    -0.0399      0.008     -5.151      0.000      -0.055      -0.025
union                     0.2021      0.030      6.672      0.000       0.143       0.262


In [6]:
t_value = result.tvalues['y85:female']  # y85:femaleのt値

dof = result.df_resid  # 自由度 n-k-1

In [7]:
1-t.cdf(t_value, dof)  # p値の計算

0.048840603301735674

In [8]:
kielmc = wooldridge.data('kielmc')

wooldridge.data('kielmc', description=True)

name of dataset: kielmc
no of variables: 25
no of observations: 321

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| year     | 1978 or 1981                    |
| age      | age of house                    |
| agesq    | age^2                           |
| nbh      | neighborhood, 1-6               |
| cbd      | dist. to cent. bus. dstrct, ft. |
| intst    | dist. to interstate, ft.        |
| lintst   | log(intst)                      |
| price    | selling price                   |
| rooms    | # rooms in house                |
| area     | square footage of house         |
| land     | square footage lot              |
| baths    | # bathrooms                     |
| dist     | dist. from house to incin., ft. |
| ldist    | log(dist)                       |
| wind     | prc. time wind incin. to house  |
| lprice   | log(price)                      |
| y81      | =1 if year == 1981       

In [9]:
formula = 'rprice ~ nearinc + y81 + nearinc:y81'

result = ols(formula, data=kielmc).fit()

In [10]:
print(result.summary().tables[1])

                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    8.252e+04   2726.910     30.260      0.000    7.72e+04    8.79e+04
nearinc     -1.882e+04   4875.322     -3.861      0.000   -2.84e+04   -9232.293
y81          1.879e+04   4050.065      4.640      0.000    1.08e+04    2.68e+04
nearinc:y81 -1.186e+04   7456.646     -1.591      0.113   -2.65e+04    2806.867


In [11]:
t_value = result.tvalues['nearinc:y81']  # t値

dof = result.df_resid  # 自由度 n-k-1

t.cdf(t_value, dof)  # p値の計算

0.0562974203495265

In [12]:
formula_1 = 'np.log(rprice) ~ nearinc * y81'

result_1 = ols(formula_1, data=kielmc).fit()

print(result_1.summary().tables[1])

                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      11.2854      0.031    369.839      0.000      11.225      11.345
nearinc        -0.3399      0.055     -6.231      0.000      -0.447      -0.233
y81             0.1931      0.045      4.261      0.000       0.104       0.282
nearinc:y81    -0.0626      0.083     -0.751      0.453      -0.227       0.102


In [13]:
formula_2 = 'np.log(rprice) ~ nearinc * y81 + age + I(age**2) + \
            np.log(intst) + np.log(land) + np.log(area) + rooms + baths'

result_2 = ols(formula_2, data=kielmc).fit()

print(result_2.summary().tables[1])

                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         7.6517      0.416     18.399      0.000       6.833       8.470
nearinc           0.0322      0.047      0.679      0.498      -0.061       0.126
y81               0.1621      0.028      5.687      0.000       0.106       0.218
nearinc:y81      -0.1315      0.052     -2.531      0.012      -0.234      -0.029
age              -0.0084      0.001     -5.924      0.000      -0.011      -0.006
I(age ** 2)    3.763e-05   8.67e-06      4.342      0.000    2.06e-05    5.47e-05
np.log(intst)    -0.0614      0.032     -1.950      0.052      -0.123       0.001
np.log(land)      0.0998      0.024      4.077      0.000       0.052       0.148
np.log(area)      0.3508      0.051      6.813      0.000       0.249       0.452
rooms             0.0473      0.017      2.732      0.007       0.013       0.081
baths           

In [14]:
# url の設定
url = 'https://raw.githubusercontent.com/Haruyama-KobeU/Haruyama-KobeU.github.io/master/data/data4.csv'

# 読み込み
df = pd.read_csv(url)

df

,year,country,gdp,inv,con,pop
0,2000,Australia,90,30.0,50.0,10
1,2001,Australia,100,40.0,80.0,11
2,2002,Australia,120,NaN,70.0,16
3,2000,Japan,100,20.0,80.0,8
4,2001,Japan,95,25.0,70.0,9
5,2002,Japan,93,21.0,72.0,10
6,2000,UK,100,30.0,70.0,11
7,2001,UK,110,39.0,71.0,12
8,2002,UK,115,55.0,NaN,14


In [15]:
df = df.sort_values(['country','year']).reset_index(drop=True)

df

,year,country,gdp,inv,con,pop
0,2000,Australia,90,30.0,50.0,10
1,2001,Australia,100,40.0,80.0,11
2,2002,Australia,120,NaN,70.0,16
3,2000,Japan,100,20.0,80.0,8
4,2001,Japan,95,25.0,70.0,9
5,2002,Japan,93,21.0,72.0,10
6,2000,UK,100,30.0,70.0,11
7,2001,UK,110,39.0,71.0,12
8,2002,UK,115,55.0,NaN,14


In [16]:
df_group = df.groupby('country')

df_group

In [17]:
df_group['gdp'].mean()

country
Australia    103.333333
Japan         96.000000
UK           108.333333
Name: gdp, dtype: float64

In [18]:
var = ['gdp', 'inv']

df_diff = df_group[var].diff()

df_diff

,gdp,inv
0,NaN,NaN
1,10.0,10.0
2,20.0,NaN
3,NaN,NaN
4,-5.0,5.0
5,-2.0,-4.0
6,NaN,NaN
7,10.0,9.0
8,5.0,16.0


In [19]:
df_diff_1 = df_diff.rename(columns={'gdp':'gdp_diff','inv':'inv_diff'})

df_diff_1

,gdp_diff,inv_diff
0,NaN,NaN
1,10.0,10.0
2,20.0,NaN
3,NaN,NaN
4,-5.0,5.0
5,-2.0,-4.0
6,NaN,NaN
7,10.0,9.0
8,5.0,16.0


In [20]:
pd.concat([df, df_diff_1], axis='columns')

,year,country,gdp,inv,con,pop,gdp_diff,inv_diff
0,2000,Australia,90,30.0,50.0,10,NaN,NaN
1,2001,Australia,100,40.0,80.0,11,10.0,10.0
2,2002,Australia,120,NaN,70.0,16,20.0,NaN
3,2000,Japan,100,20.0,80.0,8,NaN,NaN
4,2001,Japan,95,25.0,70.0,9,-5.0,5.0
5,2002,Japan,93,21.0,72.0,10,-2.0,-4.0
6,2000,UK,100,30.0,70.0,11,NaN,NaN
7,2001,UK,110,39.0,71.0,12,10.0,9.0
8,2002,UK,115,55.0,NaN,14,5.0,16.0


In [21]:
pd.merge(df, df_diff, left_index=True, right_index=True, suffixes=('', '_diff'))

,year,country,gdp,inv,con,pop,gdp_diff,inv_diff
0,2000,Australia,90,30.0,50.0,10,NaN,NaN
1,2001,Australia,100,40.0,80.0,11,10.0,10.0
2,2002,Australia,120,NaN,70.0,16,20.0,NaN
3,2000,Japan,100,20.0,80.0,8,NaN,NaN
4,2001,Japan,95,25.0,70.0,9,-5.0,5.0
5,2002,Japan,93,21.0,72.0,10,-2.0,-4.0
6,2000,UK,100,30.0,70.0,11,NaN,NaN
7,2001,UK,110,39.0,71.0,12,10.0,9.0
8,2002,UK,115,55.0,NaN,14,5.0,16.0


In [22]:
crime2 = wooldridge.data('crime2')

In [23]:
wooldridge.data('crime2', description=True)

name of dataset: crime2
no of variables: 34
no of observations: 92

+----------+----------------------------+
| variable | label                      |
+----------+----------------------------+
| pop      | population                 |
| crimes   | total number index crimes  |
| unem     | unemployment rate          |
| officers | number police officers     |
| pcinc    | per capita income          |
| west     | =1 if city in west         |
| nrtheast | =1 if city in NE           |
| south    | =1 if city in south        |
| year     | 82 or 87                   |
| area     | land area, square miles    |
| d87      | =1 if year = 87            |
| popden   | people per sq mile         |
| crmrte   | crimes per 1000 people     |
| offarea  | officers per sq mile       |
| lawexpc  | law enforce. expend. pc, $ |
| polpc    | police per 1000 people     |
| lpop     | log(pop)                   |
| loffic   | log(officers)              |
| lpcinc   | log(pcinc)                 |
| llawex

In [24]:
crime2.head()

,pop,crimes,unem,officers,pcinc,west,nrtheast,south,year,area,...,clcrimes,clpop,clcrmrte,lpolpc,clpolpc,cllawexp,cunem,clpopden,lcrmrt_1,ccrmrte
0,229528.0,17136.0,8.2,326,8532,1,0,0,82,44.599998,...,NaN,NaN,NaN,0.350872,NaN,NaN,NaN,NaN,NaN,NaN
1,246815.0,17306.0,3.7,321,12155,1,0,0,87,44.599998,...,0.009871,0.072614,-0.062743,0.262802,-0.08807,0.977952,-4.5,0.072615,4.312912,-4.540268
2,814054.0,75654.0,8.1,1621,7551,1,0,0,82,375.000000,...,NaN,NaN,NaN,0.688772,NaN,NaN,NaN,NaN,NaN,NaN
3,933177.0,83960.0,5.4,1803,11363,1,0,0,87,375.000000,...,0.104170,0.136568,-0.032398,0.658612,-0.03016,0.200762,-2.7,0.136568,4.531899,-2.962654
4,374974.0,31352.0,9.0,633,8343,1,0,0,82,49.799999,...,NaN,NaN,NaN,0.523614,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# 観察単位の数
n = len(crime2)/2

# 観察単位ID用のリスト作成 [1,2,3,4....]
lst = [i for i in range(1,int(n)+1)]

# 観察単位ID用のリスト作成 [1,1,2,2,3,3,4,4....]
country_list = pd.Series(lst).repeat(2).to_list()

In [26]:
crime2['county'] = country_list  # 追加

crime2.loc[:,['county','year']].head()  # 確認

,county,year
0,1,82
1,1,87
2,2,82
3,2,87
4,3,82


In [27]:
var = ['unem', 'crmrte']  # groupbyで差分を取る列の指定

names = {'unem':'unem_diff', 'crmrte':'crmrte_diff'}  # 差分の列のラベル

crime2_diff = crime2.groupby('county')[var].diff().rename(columns=names)

crime2_diff.head()

,unem_diff,crmrte_diff
0,NaN,NaN
1,-4.5,-4.540268
2,NaN,NaN
3,-2.7,-2.962654
4,NaN,NaN


In [28]:
formula_1 = 'crmrte_diff ~ unem_diff'

result_1 = ols(formula_1, crime2_diff).fit()

print(result_1.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     15.4022      4.702      3.276      0.002       5.926      24.879
unem_diff      2.2180      0.878      2.527      0.015       0.449       3.987


In [29]:
t_value = result_1.tvalues['unem_diff']  # t値

dof = result_1.df_resid  # 自由度 n-k-1

1-t.cdf(t_value, dof)  # p値の計算

0.007594658874412463

In [30]:
formula_ols_1 = 'crmrte ~ d87 + unem'

result_ols_1 = ols(formula_ols_1, crime2).fit()

print(result_ols_1.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     93.4202     12.739      7.333      0.000      68.107     118.733
d87            7.9404      7.975      0.996      0.322      -7.906      23.787
unem           0.4265      1.188      0.359      0.720      -1.935       2.788


In [31]:
formula_ols_2 = 'crmrte ~ unem'

result_ols_2 = ols(formula_ols_2, crime2).fit()

print(result_ols_2.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    103.2434      8.059     12.811      0.000      87.233     119.253
unem          -0.3077      0.932     -0.330      0.742      -2.159       1.543


In [32]:
crime4 = wooldridge.data('crime4')
crime4.head()

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,...,lpctymle,lpctmin,clcrmrte,clprbarr,clprbcon,clprbpri,clavgsen,clpolpc,cltaxpc,clmix
0,1,81,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.697630,...,-2.433870,3.006608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,82,0.038345,0.338111,0.433005,0.506993,5.59,0.001767,2.330254,24.874252,...,-2.449038,3.006608,-0.039376,0.154542,0.074143,0.071048,-0.003571,-0.011364,-0.032565,0.030857
2,1,83,0.030305,0.330449,0.525703,0.479705,5.80,0.001836,2.341801,26.451443,...,-2.464036,3.006608,-0.235316,-0.022922,0.193987,-0.055326,0.036879,0.038413,0.061477,-0.244732
3,1,84,0.034726,0.362525,0.604706,0.520104,6.89,0.001886,2.346420,26.842348,...,-2.478925,3.006608,0.136180,0.092641,0.140006,0.080857,0.172213,0.026930,0.014670,-0.027331
4,1,85,0.036573,0.325395,0.578723,0.497059,6.55,0.001924,2.364896,28.140337,...,-2.497306,3.006608,0.051825,-0.108054,-0.043918,-0.045320,-0.050606,0.020199,0.047223,0.172125


In [33]:
wooldridge.data('crime4', description=True)

name of dataset: crime4
no of variables: 59
no of observations: 630

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| county   | county identifier               |
| year     | 81 to 87                        |
| crmrte   | crimes committed per person     |
| prbarr   | 'probability' of arrest         |
| prbconv  | 'probability' of conviction     |
| prbpris  | 'probability' of prison sentenc |
| avgsen   | avg. sentence, days             |
| polpc    | police per capita               |
| density  | people per sq. mile             |
| taxpc    | tax revenue per capita          |
| west     | =1 if in western N.C.           |
| central  | =1 if in central N.C.           |
| urban    | =1 if in SMSA                   |
| pctmin80 | perc. minority, 1980            |
| wcon     | weekly wage, construction       |
| wtuc     | wkly wge, trns, util, commun    |
| wtrd     | wkly wge, whlesle, retail

In [34]:
# グループ化
crime4_group = crime4.groupby('county')

# 差分を計算したい変数
var = ['lcrmrte', 'lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc']

# 差分のDataFrame
crime4_diff = crime4_group[var].diff()

# DataFrameの結合
crime4 = pd.merge(crime4, crime4_diff, 
                  left_index=True, right_index=True,
                  suffixes=('','_diff'))

In [35]:
formula_2 = 'lcrmrte_diff ~ d83 + d84 + d85 + d86 + d87 + \
                            lprbarr_diff + lprbconv_diff + \
                            lprbpris_diff + lavgsen_diff + \
                            lpolpc_diff'

In [36]:
result_2 = ols(formula_2, crime4).fit()

print(result_2.summary().tables[1])

                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0077      0.017      0.452      0.651      -0.026       0.041
d83              -0.0999      0.024     -4.179      0.000      -0.147      -0.053
d84              -0.0479      0.024     -2.040      0.042      -0.094      -0.002
d85              -0.0046      0.023     -0.196      0.845      -0.051       0.042
d86               0.0275      0.024      1.139      0.255      -0.020       0.075
d87               0.0408      0.024      1.672      0.095      -0.007       0.089
lprbarr_diff     -0.3275      0.030    -10.924      0.000      -0.386      -0.269
lprbconv_diff    -0.2381      0.018    -13.058      0.000      -0.274      -0.202
lprbpris_diff    -0.1650      0.026     -6.356      0.000      -0.216      -0.114
lavgsen_diff     -0.0218      0.022     -0.985      0.325      -0.065       0.022
lpolpc_diff     